In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

# Introducción

El objetivo de la minería de opiniones es el de asociar, de forma automática, un *sentimiento* con algún comentario escrito. La minería de opiniones es una de las estrategias más utilizadas en la actualidad para extraer información de diversas fuentes dado el auge que tiene la comunicación escrita a través de la web: **correos electrónicos, comentarios en sitios web o redes sociales, blogs, discusiones en foros, etc.**

La siguiente gráfica muestra los resultados de una encuesta aplicada a diferentes asistentes (representantes de la industria y de la academia) a la conferencia del Journal of Data Analysis Techniques (2014), en la cual se solicitó a los participantes identificar las fuentes de datos textuales que utilizan con mayor frecuencia:

<img src="fuentes.png" width=400/>

La imagen corresponde al libro de *Essentials of Business Analytics* de Pochiraju.

Podemos plantear de forma progresiva el objetivo de la minería de opiniones, limitando los sentimientos que se pueden asociar a un comentario, reduciéndolos por ejemplo a tres categorías:

$$\{\text{negativo}, \text{neutro}, \text{positivo}\}$$

Con estas etiquetas, buscamos idenificar la **polaridad** de un comentario. Aún así, lograr el objetivo de la minería de opiniones puede ser un gran desafío. Considera el siguiente comentario:

        "Compré un IPhone hace algunos días. Resultó ser un excelente teléfono. La pantalla táctil es fantástica. El sonido es muy claro. Sin embargo, mi madre se molestó porque lo compré sin pedirle permiso. Ella piensa que el teléfono es muy caro y me ha pedido que lo regrese a la tienda."
        
**¿Qué polaridad le asignarías?**

## Planteamiento del problema

Para ganar claridad, podemos plantear el problema de manera formal. Dado un fragmento de texto (comentario) buscamos identificar los siguientes elementos:

$$(e, a, oo, s, t)$$

* $e$: es la entidad de la cual se opina (por ejemplo del IPhone).
* $a$: atributo de la entidad (pantalla táctil o sonido).
* $oo$: orientación de la opinión (positiva, negativa o neutra).
* $s$: sujeto que emite la opinión (el comprador del teléfono o su mamá).
* $t$: momento en el que se emite la opinión.


En este curso exploraremos dos estrategias para abordar el problema de la minería de opiniones. La primera de ellas plantea el problema como uno de **aprendizaje automático** y la segunda se basa en el uso de **lexicones**.


# Minería de opiniones con aprendizaje supervisado

Para poder aplicar esta estrategia es necesario disponer de un conjunto de comentarios para los cuales se ha identificado la polaridad. En las aplicaciones reales, puede ser necesario realizar la identificación de la polaridad manualmente, lo cual representa una desventaja importante debido a que para obtener resultados confiables puede ser necesario construir un **conjunto de entrenamiento** con decenas de miles de observaciones.  

Esta estrategia se basa en la representación de cada comentario como un vector de características, específicamente a través de la construcción de la **matriz TF (frecuencia de de término) o de la TF-IDF** (frecuencia de término y frecuencia inversa de documento). Una vez que se ha construido esta matriz, puede construirse un modelo utilizando alguna técnica de clasificación; **naive Bayes** y las **máquinas de soporte vectorial** son dos de las más utilizadas.

En las siguientes celdas se muestra un ejemplo de la aplicación de las máquinas de soporte vectorial sobre un conjunto de comentarios.



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [ ]:
comments = pd.read_csv("amazon_baby.csv")
comments.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


Considerando la calificación que el usuario otorgó en cada comentario (rating) podemos identificar la polaridad de cada comentario, de manera que etiquetamos como *positivos* a los que tienen una calificación mayor a 3 y *negativos* los que tienen una calificación menor a 3. Los calificados con 3 pueden considerarse *neutrales*, pero para este ejemplo se ignorarán.

In [ ]:
df = comments.loc[comments["rating"] != 3]
df["polaridad"] = df['rating'].apply(lambda x: 1 if x > 3 else -1)
df = df.dropna()
df = df.sample(n = 10000)
df.head()

<ipython-input-7-7b6ebfc0a16c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["polaridad"] = df['rating'].apply(lambda x: 1 if x > 3 else -1)


,name,review,rating,polaridad
18561,Fisher-Price Ocean Wonders Mobile with Remote ...,I bought this mobile when my daughter was only...,5,1
159985,Adiri NxGen Stage 2 Nurser Medium Flow Baby Bo...,"The nipple is weird my baby didn't like it, if...",1,-1
161109,Bugaboo Mosquito Net,I purchased this for my bugaboo bee and it fit...,5,1
78694,"Graco Lauren Classic Crib, Espresso",Easy to install. Nice white color and sturdy. ...,5,1
149171,Summer Infant 28034/28035 Best View Extra Adaptor,My Charger died for my son's monitor and I tho...,5,1


La tabla de datos original contiene poco menos de 200 mil comentarios, pero para esta aplicación consideraremos una muestra de 10 mil.

Considera que la mayoría de los comentarios tienen una polaridad positiva (84%):

In [ ]:
len(df.loc[df["polaridad"] > 0])/len(df)

0.8365

## Preproceso

Vectorizamos los comentarios y separamos conjuntos de entrenamiento y de prueba. Además limitamos el total de términos a los 100 más frecuentes.

In [ ]:
corpus = df["review"]
vectorizer = TfidfVectorizer(stop_words="english", max_features=100)
X = vectorizer.fit_transform(corpus)
y = df["polaridad"]

In [ ]:
X.shape

(10000, 100)

In [ ]:
# 70% de las observaciones se usan para construir el modelo
# el restante 30% para probar
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Construimos un modelo basándonos en una máquina de soporte vectorial para clasificar los comentarios:

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {'C': [1, 10, 1e2, 1e3], 'gamma': [0.01, 0.001, 0.0001]}

clf = GridSearchCV(
    SVC(kernel='rbf', class_weight='balanced'), param_grid
)
clf = clf.fit(X_train, y_train)
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Best estimator found by grid search:
SVC(C=10, class_weight='balanced', gamma=0.0001)


In [ ]:
y_predicted = clf.predict(X_test)

#exactitud
sum(y_predicted == y_test)/y_test.shape[0]

0.7346666666666667

In [ ]:
#otras métricas
from sklearn import metrics

print(f"Reporte del clasificador:\n"
      f"{metrics.classification_report(y_test, y_predicted)}\n")

Reporte del clasificador:
              precision    recall  f1-score   support

          -1       0.35      0.79      0.49       477
           1       0.95      0.72      0.82      2523

    accuracy                           0.73      3000
   macro avg       0.65      0.76      0.65      3000
weighted avg       0.85      0.73      0.77      3000




## Naive Bayes

Otra técnica que se aplica con mucho éxito en la minería de opiniones (y en la clasificación de documentos en general) es la de Naive Bayes.

Suponemos que observamos un vector de características, $X$, (vector de frecuencia de términos por ejemplo), nos interesamos en evaluar la probabilidad condicionada $$P(Y \;|\; X)$$ para cada una de las clases, en nuestro ejemplo, deberíamos evaluar $P(Y =1\;|\; X)$ y $P(Y = -1 \;|\; X)$.

Una vez que evaluamos las probabilidades, para clasificar a un comentario, solamente debemos comparar $P(Y =1\;|\; X)$ y $P(Y = -1 \;|\; X)$.

La evaluación se realiza a través de la regla de Bayes:

$$P(Y \; | \; X) = \frac{P(X \; | \; Y)P(Y)}{P(X)}$$

Más aún, en la técnica de Naive Bayes, suponemos que las características son independientes una vez que se conoce la clase, esto es, podemos evaluar la **verosimilitud** son la siguiente expresión

$$P(X \;|\; Y) = P(x_1, x_2, \ldots, x_d \;|\;Y) = \prod_{i=1}^d P(x_i\;| \; Y)$$

Evaluar estas probabilidades es muy sencillo, solamente debemos contar cuántas veces aparece el término $x_i$ en los comentarios asociados a cada clase.

### Ejemplo

Por ejemplo, supongamos que utilizamos los siguientes comentarios acerca de un artículo para la aplicación del método de Naive Bayes:

1. "La calidad es excelente" ($Y=1$)
2. "El artículo es muy bueno en general, pero el diseño sobresale, es excelente" ($Y=1$)
3. "No es malo, es pésimo" ($Y=-1$)
4. "No me satisface" ($Y=-1$)
5. "El artículo es muy malo" ($Y=-1$)


Tenemos las siguientes probabilidades:

1. $P(Y=1) = \frac{2}{5}$
2. $P(Y=-1) = \frac{3}{5}$

Supongamos que los términos que componen el vocabulario son: $$\{\text{excelente}, \text{bueno}, \text{pésimo}, \text{satisface}, \text{malo}\}$$

Entonces:

1. $P(\text{bueno} \;|\; Y = 1) = \frac{1}{2}$
2. $P(\text{excelente} \;|\; Y = 1) = 1$
3. $P(\text{pésimo} \;|\; Y = -1) = \frac{1}{3}$
4. $P(\text{malo} \;|\; Y = -1) = \frac{2}{3}$

Podemos evaluar las probabilidades $P(Y=1\;|\;X)$ y $P(Y=-1\;|\;X)$ para un nuevo comentario, como, "Me parece excelente":

\begin{align}
    P(Y=1\;&|\;\text{excelente}=1, \text{bueno}=0, \text{pésimo}=0, \text{satisface}=0, \text{malo}=0)\\ &= P(\text{excelente}\;|\; Y=1)\times P(\sim \text{bueno}\;|\; Y=1)\times \cdots\\ &\times P(\sim \text{malo}\;|\; Y=1)\times P(Y=1)/k\\
    &=1\times\frac{1}{2}\times 1 \times 1 \times 1 \times \frac{2}{5} /k
\end{align}



## Suavizado de Laplace

Cabe señalar que, de aplicarse el método de Naive Bayes tal como está definido, es muy probable que varias probabilidades tomen el valor de cero; por ejemplo, si el comentario fuera **El producto me satisface, es excelente**, entonces $P(Y = 1 \;|\; X) = 0$ dado que $P(\text{satisface} \;|\; Y = 1) = 0$ (la palabra *satisface* no aparece en los comentarios con polaridad positiva):

$$P(Y = 1 \;|\; X) = 1\times \frac{1}{2} \times 1 \times \color{red}{0} \times 1 \times \frac{2}{5}/k = 0$$

Para evitar este problema, modificamos la evaluación de las probabilidades usando el criterio de suavizado de Laplace:

$$P(X_i \; | \; Y=y_j) = \frac{(\text{# documentos en de la clase } y_j \text{ en donde aparezca } X_i)+\alpha}{N_{y_j}+\alpha d}$$

Observa que con esta condición, si $\alpha=0.01$:

$$P(\text{satisface} \; | \; Y=1) = \frac{0+\alpha}{2+\alpha\times 5} = 0.0049$$

De esta forma, se tendría:

$$P(Y = 1 \;|\; X) = 1\times \frac{1}{2} \times 1 \times \color{red}{0.0048} \times 1 \times \frac{2}{5}/k$$

Como ejercicio, evalúa $P(Y=-1 \; | \; X)$



In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Cargar los datos desde el archivo CSV
data = pd.read_csv("amazon_baby.csv")

# Explorar los primeros registros del conjunto de datos
print(data.head())

# Preprocesamiento: Eliminar filas con valores nulos en la columna 'review' y asignar etiquetas a las opiniones
data = data.dropna(subset=['review'])
data['sentiment'] = data['rating'].apply(lambda rating: 1 if rating > 3 else -1)

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['sentiment'], test_size=0.2, random_state=42)

# Vectorización de texto utilizando CountVectorizer
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Entrenar un clasificador Naive Bayes
classifier = MultinomialNB()
classifier.fit(X_train_vectorized, y_train)

# Realizar predicciones en el conjunto de prueba
predictions = classifier.predict(X_test_vectorized)

# Calcular la precisión del clasificador
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

# Calcular la probabilidad condicional P(Y = -1 | X)
# Supongamos que queremos calcular la probabilidad para una revisión específica, por ejemplo, la primera revisión en el conjunto de prueba
specific_review = X_test.iloc[0]
specific_review_vectorized = vectorizer.transform([specific_review])
probability_negative = classifier.predict_proba(specific_review_vectorized)[:, 0]

print("Probability P(Y = -1 | X):", probability_negative)


                                                name  \
0                           Planetwise Flannel Wipes   
1                              Planetwise Wipe Pouch   
2                Annas Dream Full Quilt with 2 Shams   
3  Stop Pacifier Sucking without tears with Thumb...   
4  Stop Pacifier Sucking without tears with Thumb...   

                                              review  rating  
0  These flannel wipes are OK, but in my opinion ...       3  
1  it came early and was not disappointed. i love...       5  
2  Very soft and comfortable and warmer than it l...       5  
3  This is a product well worth the purchase.  I ...       5  
4  All of my kids have cried non-stop when I trie...       5  
Accuracy: 0.8549027120221121
Probability P(Y = -1 | X): [5.49229354e-06]
